# Joint Solar Orbiter, Parker Solar Probe, and DKIST Meeting
### April 9 2024
Given by Laura A. Hayes


![title](./images/sunpy_logo.png)


This notebook is to demonstrate how you can work with different data within the SunPy ecosystem, with a specific focus on Solar Orbiter observations.

It serves to demonstate how the SunPy Ecosystem allows you to easily work different types of data using the same set of tools.

Here, I'll demonstrate how you can search for and download Solar Orbiter data. We'll also then look at an event analysis workflow for a large solar eruptive event that occured on Sept 5 2022 by both PSP and Solar Orbiter.

Some useful links:

* An introduction to SunPy https://docs.sunpy.org/en/stable/tutorial/index.html
* Solar Orbiter Archive https://soar.esac.esa.int/soar/
* Summary of SOOPs that have been run: https://www.cosmos.esa.int/web/solar-orbiter/soops-summary
* Inventory plots of data on SOAR: https://www.cosmos.esa.int/web/soar/inventory-plots
* DKIST Python Tools https://docs.dkist.nso.edu/projects/python-tools/en/stable/
* STIX data center: https://datacenter.stix.i4ds.net/stix
* EPD data loader: https://github.com/jgieseler/solo-epd-loader


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.visualization import ImageNormalize, AsinhStretch
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.visualization import quantity_support
quantity_support()

from sunpy.net import Fido, attrs
import sunpy.map
from sunpy import timeseries as ts
from sunpy.coordinates import get_earth, get_horizons_coord
import sunpy.visualization.drawing
from sunkit_instruments import iris

import sunraster.instr.spice
import dkist

import sunpy_soar
import dkist.net

import pathlib
import warnings

import astropy.io.fits
import astropy.wcs
from astropy.nddata import Cutout2D

from sunpy.coordinates import propagate_with_solar_surface

# 1. Searching for and downloading data

## Overview of sunpy's Fido Unified Downloader
Fido is sunpy's interface for searching and downloading solar physics data.
It offers a unified interface for searching and fetching data irrespective of the underlying client or webservice from where the data is obtained.
You can also search and accesses multiple instruments and all available data providers in a single query.
It supplies a single, easy, consistent and extendable way to get most forms of solar physics data the community need.

For more information about Fido and how to use it check out the documentation on our website: https://docs.sunpy.org/en/stable/tutorial/acquiring_data/index.html

Fido offers access to data available through:

* VSO
* JSOC (through drms)
* Individual data providers from web accessible sources (http, ftp, etc)
* CDAWeb
* HEK
* HELIO
  
As described here Fido provides access to many sources of data through different clients, these clients can be defined inside sunpy or in other packages 

For example:

- DKIST data can be accessed using Fido through [DKIST User Tools](https://docs.dkist.nso.edu/projects/python-tools/en/latest/tutorial/2_search_and_asdf_download.html).
- Solar Orbiter data can be access from the [Solar Orbiter Archive](https://soar.esac.esa.int/soar/) from [sunpy_soar](https://docs.sunpy.org/projects/soar/)


#### Importantly, Solar Orbiter data can be accessed through the client defined in the `sunpy_soar` affiliated package. 
The SOAR client is registered once we install `sunpy_soar` above. Without installing it, it wont be registered within Fido.

Lets first inspect the clients that are available through Fido:

In [ ]:
Fido

In [ ]:
attrs.Provider

There are also Solar Orbiter specific attributes that are registered when we import `sunpy_soar`, such as `a.soar.Product`, and `a.soar.SOOP` (Solar Orbiter Observing Plan).

What this allows you to do is to construct a query based on these attributes, for example, a specific data product during a specific SOOP etc.

In [ ]:
attrs.soar.Product

In [ ]:
attrs.soar.SOOP

## How to construct a query

To search for data with Fido, you need to specify attributes to search with. These are usually a timerange (`a.Time`), and an instrument (`a.Instrument`). But you can also add lots of different attributes to make your query more specific. It should also be noted that Fido will return all possible results given the conditions of your query, so for example, if two data providers host the same data, it will return two results (for example the VSO provides the EUI data as well as the SOAR). You can also specifiy the provider you want to search for in your query.

In [ ]:
res_eui = Fido.search(attrs.Time("2022-03-25", "2022-03-26"), 
                      attrs.Instrument("EUI"), 
                      attrs.Provider.soar)

this returns an [`UnifiedResponse`](https://docs.sunpy.org/en/stable/generated/api/sunpy.net.fido_factory.UnifiedResponse.html#sunpy.net.fido_factory.UnifiedResponse) object (like a table) containing all the search results that match the search attributes. We haven't downloaded anything here, we've just found the results of the query. Lets inspect the table and see whats there:

In [ ]:
res_eui

We can also construct a query to search for SWA data (i.e. in-situ data). We can also specific the level data that we want, here Level 2 data. 

(Note that there's no SOOP name associated in the returned table - thats because the in-situ data products do not have this tied to their data)

In [ ]:
res_swa = Fido.search(attrs.Time("2022-03-25", "2022-03-26"), 
                      attrs.Instrument("SWA"), 
                      attrs.Provider.soar)

Say for example, instead we're just interested in data from a particular SOOP. Checkout the [SOOP summary page](https://www.cosmos.esa.int/web/solar-orbiter/soops-summary) for more details.
Lets search for the coronal dynamics SOOP, which focuses on observing structures in the outer corona and linking them to the heliosphere observed in-situ.

In [ ]:
res_soop = Fido.search(attrs.Time("2022-03-25", "2022-03-25 01:00"), 
                       attrs.soar.SOOP.l_full_hres_hcad_coronal_dynamics, 
                       attrs.Provider.soar)

In [ ]:
res_soop

## Downloading the data

### To download the data files you've found from your query, you can use `Fido.fetch`

Now that we have located the files were interested in via a `Fido.search`, we can download them via `Fido.fetch`.
You pass your query results from `Fido.search` to the `.fetch`. You can also specifiy the path for which to save them locally. Here lets just save them in our current working directory. If you do not pass anything it will save them location set in the sunpy config file.


Lets make a smaller search for some EUI data over some timerange

In [ ]:
res_eui_soop = Fido.search(attrs.Time("2022-03-25", "2022-03-25 01:00"), 
                           attrs.soar.SOOP.l_full_hres_hcad_coronal_dynamics, 
                           attrs.Level(2), attrs.Provider.soar, attrs.Instrument("EUI"))

In [ ]:
Fido.fetch(res_eui_soop, path="./data")

We can pass these files directly to a sunpy map that reads the information in the metadata and constructs a `sunpy.map.Map`

In [ ]:
eui_map = sunpy.map.Map('data/solo_L2_eui-fsi304-image_20220325T000815304_V02.fits')

In [ ]:
eui_map.plot()

Similarly we can search for in-situ data - say here Solar Orbiter Magnetometer data

In [ ]:
res_mag = Fido.search(attrs.Time("2022-03-25", "2022-03-25 23:00"),
                      attrs.Instrument.mag, 
                      attrs.soar.Product('mag-rtn-normal-1-minute'), 
                      attrs.Level(2))

In [ ]:
Fido.fetch(res_mag, path="./data")

We can then pass these files directly into sunpy.timeseries.TimeSeries and plot

In [ ]:
solo_mag_ts = ts.TimeSeries('data/solo_L2_mag-rtn-normal-1-minute_20220325_V01.cdf')
solo_mag_ts.plot(columns=["B_RTN_0", "B_RTN_1", "B_RTN_2"])

# Accessing data from the CDAWeb with sunpy - which is very helpful for in-situ data

There is also a CDAWeb client within sunpy. CDAWeb data can be accessed when the `cdaweb.Dataset` attribute is provided to the search.

The data available from the SOAR is also available from the CDAWeb. You may be used to working with this (especially if you mainly work with in-situ observations), so lets go through how the data can also be accessed this way. This is handy, as you can also access many other in-situ measurements from this too.

In [ ]:
# attrs.cdaweb.Dataset

So for example, we can also query for data (e.g. Solar Orbiter Magnetometer data) from the CDAWeb

In [ ]:
res_cdaw = Fido.search(attrs.Time("2022-03-25", "2022-03-26"), 
                       attrs.cdaweb.Dataset('SOLO_L2_MAG-RTN-NORMAL-1-MINUTE'))

In [ ]:
res_cdaw

In [ ]:
Fido.fetch(res_cdaw, path="./data")

In [ ]:
solo_mag_ts = ts.TimeSeries("data/solo_l2_mag-rtn-normal-1-minute_20220325_v01.cdf")
solo_mag_ts.plot(columns=["B_RTN_0", "B_RTN_1", "B_RTN_2"])

# Example workflow for analysing data - Solar Orbiter RSW 7 Oct 2022

Here, we'll do an example workflow of using functionality within the SunPy ecosystem to analyse some data from remote sensing window (RSW) 5 of Solar Orbiter during the long term active region tracking `R_SMALL_MRES_MCAD_AR-Long-Term` SOOP. We'll use Fido to query sunpy soar, and also search for coordinated data from Earth and DKIST.

This example workflow will use:
- sunpy core
- sunraster
- dkist


### Lets plot the locations of the spacecraft

Lets first plot the locations of the spacecraft at the time of interest - which we'll seta t 2022-10-24 19:00

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.subplot(projection='polar')

# Plot the Sun
ax.plot(0, 0, marker='o', markersize=20, label='Sun', color='yellow')

# Plot the satellite locations
obstime = "2022-10-24 19:00"
for body_name in ['Earth', 'Solar Orbiter', "PSP"]:
    if body_name == 'Earth':
        body = get_earth(obstime)
    else:
        body = get_horizons_coord(body_name, time=obstime)
    p, = ax.plot(body.lon.to('rad'), body.radius.to(u.AU), 'o', label=body_name)
    ax.plot([body.lon.to_value('rad'), 0], [body.radius.to_value(u.AU), 0], ls='--', color=p.get_color())

ax.set_theta_zero_location("S")
ax.set_rlabel_position(90)
ax.set_rlim(0, 1.3)
ax.legend()


In [ ]:
time_range = attrs.Time("2022-10-24T18:55", "2022-10-24T19:35")

In [ ]:
Fido.search(time_range, attrs.Instrument("PHI"))

## search for Solar Orbiter data

In [ ]:
Fido.search(
    time_range,
    attrs.soar.Product('EUI-HRIEUV174-IMAGE') | attrs.soar.Product('EUI-FSI174-IMAGE') |
    attrs.soar.Product('SPICE-N-RAS') | attrs.soar.Product("PHI-HRT-BLOS"),
    attrs.Level(2)
)

## DKIST

Lets search for DKIST VISP data, as we've imported dkist.net above, the dkist client was registered

In [ ]:
Fido.search(
    time_range,
    attrs.Instrument('VISP')
)

## Combine queries all together!

In [ ]:
aia_query = attrs.Instrument.aia & attrs.Wavelength(171*u.Angstrom) & attrs.Sample(10*u.minute)

solo_query = (attrs.soar.Product('EUI-HRIEUV174-IMAGE') | attrs.soar.Product('EUI-FSI174-IMAGE') |
              attrs.soar.Product('SPICE-N-RAS') | attrs.soar.Product("PHI-HRT-BLOS")) & attrs.Level(2)

dkist_query = attrs.Instrument("VISP")

In [ ]:
query = Fido.search(time_range, aia_query | solo_query | dkist_query)

In [ ]:
files = Fido.fetch(query, path='data/{instrument}')

In [ ]:
query2 = Fido.search(time_range, attrs.Instrument.iris, attrs.Wavelength(1330*u.angstrom))

In [ ]:
query2

In [ ]:
files = Fido.fetch(query2, path='data/{instrument}')

## Visulise the fields of view

In [ ]:
aia_map = sunpy.map.Map("./data/AIA/aia_lev1_171a_2022_10_24t19_15_09_35z_image_lev1.fits")

In [ ]:
fsi_map = sunpy.map.Map("./data/EUI/solo_L2_eui-fsi174-image_20221024T191050177_V01.fits")

In [ ]:
hri_map = sunpy.map.Map("./data/EUI/solo_L2_eui-hrieuv174-image_20221024T191500172_V01.fits")

In [ ]:
phi_map = sunpy.map.Map("./data/PHI/solo_L2_phi-hrt-blos_20221024T191503_V01.fits")

In [ ]:
raster_spice = sunraster.instr.spice.read_spice_l2_fits('./data/SPICE/solo_L2_spice-n-ras_20221024T191303_V05_150995395-059.fits')
spice_window = raster_spice['N IV 765 - SH - Comp 8 ... Ne VIII 770 - LH - Comp 8 (Merged)']

In [ ]:
iris_map = iris.SJI_to_sequence("./data/IRIS/iris_l2_20221024_190447_3643101203_sji_1330_t000_fits.gz")

In [ ]:
visp = dkist.load_dataset("./data/VISP/VISP_L1_20221024T185745_BKEWK.asdf")

## Lets inspect the map from Solar Orbiter EUI/FSI

In [ ]:
fsi_map.plot()

In [ ]:
fsi_map_zoom = fsi_map.submap(SkyCoord(-3000,-3000, unit='arcsec', frame=fsi_map.coordinate_frame),
                              top_right=SkyCoord(3000, 3000, unit='arcsec', frame=fsi_map.coordinate_frame))

In [ ]:
fsi_map_zoom.plot()

In [ ]:
fsi_map_zoom_rotate = fsi_map_zoom.rotate(missing=fsi_map_zoom.data.min())

In [ ]:
fsi_map_zoom_rotate.plot()

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=fsi_map_zoom)
fsi_map_zoom.plot(axes=ax)
fsi_map_zoom.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(hri_map.dimensions),
    label='HRI', edgecolor='C0', lw=2,
    transform=ax.get_transform(hri_map.wcs),
)

In [ ]:
visp_frame = visp.wcs.output_frame.frames[1].reference_frame
visp_space = visp[0, :, 500, :]
visp_corners = visp_space.wcs.pixel_to_world([0, visp_space.data.shape[1]-1],[0, visp_space.data.shape[0]-1])[0]


In [ ]:
spice_map = sunpy.map.Map(spice_window[0,51,:,:].data, spice_window[0,51,:,:].meta)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=fsi_map_zoom)
fsi_map_zoom.plot(axes=ax)


fsi_map_zoom.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(hri_map.dimensions),
    label="HRI",
    lw=2, edgecolor="C0",
    transform=ax.get_transform(hri_map.wcs),
)

fsi_map_zoom.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(phi_map.dimensions),
    label="PHI",
    lw=2, edgecolor='C1',
    transform=ax.get_transform(phi_map.wcs),
)

fsi_map_zoom.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(spice_map.dimensions),
    label="SPICE",
    lw=2, edgecolor='C2',
    transform=ax.get_transform(spice_map.wcs),
)

fsi_map_zoom.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(iris_map[0].dimensions),
    label="IRIS",
    lw=2, edgecolor='C6',
    transform=ax.get_transform(iris_map[0].wcs),
)

fsi_map_zoom.draw_quadrangle(
    visp_corners,
    label="VISP",
    edgecolor='white',
    lw=1,
    transform=ax.get_transform(visp_frame)
)

aia_map.draw_limb(color='r', lw=2, label="Limb from AIA")
plt.legend()


In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=aia_map)
aia_map.plot(axes=ax)


aia_map.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(hri_map.dimensions),
    label="HRI",
    lw=2, edgecolor="C0",
    transform=ax.get_transform(hri_map.wcs),
)

aia_map.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(phi_map.dimensions),
    label="PHI",
    lw=2, edgecolor='C1',
    transform=ax.get_transform(phi_map.wcs),
)

aia_map.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(spice_map.dimensions),
    label="SPICE",
    lw=2, edgecolor='C2',
    transform=ax.get_transform(spice_map.wcs),
)

aia_map.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(iris_map[0].dimensions),
    label="IRIS",
    lw=2, edgecolor='C6',
    transform=ax.get_transform(iris_map[0].wcs),
)

aia_map.draw_quadrangle(
    visp_corners,
    label="VISP",
    edgecolor='white',
    lw=1,
    transform=ax.get_transform(visp_frame)
)

fsi_map_zoom.draw_limb(color='r', lw=2, label="Limb from FSI")
plt.legend()


In [ ]:
center = SkyCoord(Tx=930*u.arcsec, Ty=630*u.arcsec, frame=hri_map.coordinate_frame)
width = 350*u.arcsec
height = 250*u.arcsec
loop_fov = center.spherical_offsets_by(width/[-2, 2], height/[-2, 2])

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(1, 3, 1, projection=hri_map)
ax2 = fig.add_subplot(1, 3, 2,projection=phi_map)
ax3 = fig.add_subplot(1, 3, 3,projection=spice_map)
hri_map.plot(axes=ax1)
hri_map.draw_quadrangle(loop_fov, axes=ax1)
phi_map.plot(axes=ax2, vmin=-500, vmax=500)
hri_map.draw_quadrangle(loop_fov, axes=ax2)
spice_map.plot(axes=ax3, aspect="auto", cmap="viridis")
hri_map.draw_quadrangle(loop_fov, axes=ax3)

In [ ]:
# fig = plt.figure()
# visp.plot(plot_axes=[None, None, 'x', None], fig=fig)
# plt.show()

## Example of reading cutout portions of the HRI data

In [ ]:
hri_maps = []
for filename in sorted(pathlib.Path('./data/EUI/').glob('solo_L2_eui-hrieuv174-image_*.fits')):
    with astropy.io.fits.open(filename) as hdul:
        with warnings.catch_warnings():  # silence some astropy FITS warnings
            warnings.simplefilter('ignore', astropy.wcs.FITSFixedWarning)
            wcs = astropy.wcs.WCS(hdul[1].header)
        with propagate_with_solar_surface():  # transform with solar rotation
            cutout = Cutout2D(hdul[1].section,  # cutout from full-image
                              position=center,
                              size=(height, width),
                              wcs=wcs)
    hri_maps.append(sunpy.map.Map(cutout.data, cutout.wcs))  # create sunpy map

In [ ]:
hri_maps = sunpy.map.Map(hri_maps, sequence=True)

In [ ]:
ani = hri_maps.plot(cmap=hri_map.plot_settings['cmap'],
                    norm=ImageNormalize(vmin=5e2, vmax=1.75e4,
                                        stretch=hri_map.plot_settings['norm'].stretch))
ani.save('eui-hri-loops.mp4', fps=15, dpi=300)

In [ ]:
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="60%" controls>
      <source src="eui-hri-loops.mp4" type="video/mp4">
</video>
</div>""")

## 3. Another date and event for an example combining PSP + Solar Orbiter in-situ data

In [ ]:
timerange_sept = attrs.Time("2022-09-05 14:30", "2022-09-05 21:30")

In [ ]:
eui_query = Fido.search(timerange_sept, 
                        attrs.soar.Product("eui-fsi174-image") , 
                        attrs.Level(2))

In [ ]:
Fido.fetch(eui_query, path="./data/{instrument}")

In [ ]:
tstart = timerange_sept.start
solo_coord = get_horizons_coord("solo", tstart)
psp_coord =  get_horizons_coord("psp", tstart)
earth_coord =  get_earth(tstart)

fig = plt.figure()
ax = fig.add_subplot(projection="polar")

ax.scatter(psp_coord.lon.to('rad'), psp_coord.radius.to(u.AU),
           label='PSP', lw=0.5)
ax.scatter(solo_coord.lon.to('rad'), solo_coord.radius.to(u.AU),
           label='Solar Orbiter', lw=0.5)
ax.scatter(earth_coord.lon.to('rad'), earth_coord.radius.to(u.AU),
           label='Earth', color='g')
ax.plot(0, 0, marker='o', ms=10, color='yellow')


# lets also get the positions over 4 days to see the perihelion of PSP
# when plotted here, you can't see Solar Orbiter move too much (dot plotted is bigger than movement)
psp_seq = get_horizons_coord("psp", tstart+np.arange(-2, 2, 0.1)*u.day)
solo_seq = get_horizons_coord("solo", tstart + np.arange(-2, 2, 0.1)*u.day)
for coord in [psp_seq, solo_seq]:
    ax.plot(coord.lon.to('rad'), coord.radius.to(u.AU))

ax.set_theta_zero_location("S")
ax.set_rlabel_position(90)
ax.set_title("Spacecraft positions {:s}".format(tstart.strftime("%Y-%m-%d")))
ax.legend(loc="upper left")

In [ ]:
eui_map_174 = sunpy.map.Map("./data/EUI/solo_L2_eui-fsi174-image_20220905T161055282_V01.fits")


In [ ]:
eui_map_174 = eui_map_174.rotate()

In [ ]:
eui_map_174_zoom = eui_map_174.submap(SkyCoord(-1800, -1800, unit='arcsec', frame=eui_map_174.coordinate_frame),
                              top_right=SkyCoord(1800, 1800, unit='arcsec', frame=eui_map_174.coordinate_frame))

In [ ]:
eui_map_174_zoom.plot()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection=eui_map_174_zoom)
eui_map_174_zoom.plot()
eui_map_174_zoom.draw_quadrangle(SkyCoord(400*u.arcsec, -750*u.arcsec, frame=eui_map_174_zoom.coordinate_frame),
                                 top_right=SkyCoord(900*u.arcsec, -400*u.arcsec, frame=eui_map_174_zoom.coordinate_frame), edgecolor='r')

# Lets see what this looks like from PSP point of view

We can use the coordinates framework with sunpy to reproject images from one viewpoint to another.
To learn more about this, check out our documentation, and also [this](https://docs.sunpy.org/en/stable/generated/gallery/map_transformations/reprojection_different_observers.html#sphx-glr-generated-gallery-map-transformations-reprojection-different-observers-py) example in the example gallery.

In [ ]:
ref_coord = SkyCoord(0*u.arcsec, 0*u.arcsec,
                     frame='helioprojective', 
                     obstime=eui_map_174_zoom.date, 
                     observer=get_horizons_coord("psp", eui_map_174_zoom.date))

# Create a FITS WCS header for the reference coordinate and frame
header = sunpy.map.make_fitswcs_header((3000, 3000),
                                        ref_coord,
                                        scale=[16, 16]*u.arcsec/u.pix,
                                      )

In [ ]:
eui_psp_view = eui_map_174_zoom.reproject_to(header)

In [ ]:
eui_psp_view.plot()
eui_psp_view.draw_limb(color='k')
eui_map_174_zoom.draw_limb(color='r')
eui_map_174_zoom.draw_quadrangle(SkyCoord(400*u.arcsec, -750*u.arcsec, frame=eui_map_174_zoom.coordinate_frame),
                                 top_right=SkyCoord(900*u.arcsec, -400*u.arcsec, frame=eui_map_174_zoom.coordinate_frame), edgecolor='r')

In [ ]:
time_range_insitu = attrs.Time("2022-09-04", "2022-09-08")
res_solo_mag = Fido.search(time_range_insitu, 
                           attrs.cdaweb.Dataset('SOLO_L2_MAG-RTN-NORMAL-1-MINUTE'))
f_solomag = Fido.fetch(res_solo_mag, path="./")
solo_mag = sunpy.timeseries.TimeSeries(f_solomag, concatenate=True)

In [ ]:
fig = plt.figure()
solo_mag.plot(columns=["B_RTN_0", "B_RTN_1", "B_RTN_2"])


In [ ]:
res_solo_swa = Fido.search(time_range_insitu, 
                           attrs.cdaweb.Dataset.solo_l2_swa_pas_grnd_mom)
f_soloswa = Fido.fetch(res_solo_swa, path="./")
solo_swa_pas = sunpy.timeseries.TimeSeries(f_soloswa, concatenate=True)

In [ ]:
fig = plt.figure()
solo_swa_pas.plot(columns=["N"])

In [ ]:
fig = plt.figure()
solo_swa_pas.plot(columns=["T"])

## Lets also look for some PSP FIELDS/MAG data

In [ ]:
result_psp_mag = Fido.search(time_range_insitu, 
                             attrs.cdaweb.Dataset.psp_fld_l2_mag_rtn_1min)
f_mag_psp = Fido.fetch(result_psp_mag, path="./")

psp_mag = sunpy.timeseries.TimeSeries(f_mag_psp, concatenate=True)

In [ ]:
clean_psp = psp_mag._data[psp_mag._data["psp_fld_l2_quality_flags"].isnull()]
psp_event = sunpy.timeseries.TimeSeries(clean_psp, psp_mag.meta, psp_mag.units)
fig = plt.figure()
psp_event.plot(columns=['psp_fld_l2_mag_RTN_1min_0', 
                        'psp_fld_l2_mag_RTN_1min_1',
                        'psp_fld_l2_mag_RTN_1min_2'])


In [ ]:
fig, ax = plt.subplots(5, figsize=(8, 10), sharex=True)

psp_event.plot(columns=['psp_fld_l2_mag_RTN_1min_0', 
                        'psp_fld_l2_mag_RTN_1min_1',
                        'psp_fld_l2_mag_RTN_1min_2'], 
              axes=ax[0])
solo_mag.plot(columns=['B_RTN_0', 'B_RTN_1', 'B_RTN_2'], axes=ax[1])
solo_swa_pas.plot(columns=['T'], axes=ax[2])
solo_swa_pas.plot(columns=['N'], axes=ax[3])
solo_swa_pas.plot(columns=['V_RTN_0', 'V_RTN_1', 'V_RTN_2'], axes=ax[4])
ax[4].set_xlabel("Time")

for aa in ax:
    aa.legend(loc="lower left")
    # aa.axvline(flare_time.datetime, color='r')

ax[0].text(0.02, 0.90, "a. PSP FIELDS/MAG", transform=ax[0].transAxes)
ax[1].text(0.02, 0.90, "b. Solar Orbiter/MAG", transform=ax[1].transAxes)
ax[2].text(0.02, 0.90, "c. Solar Orbiter/SWA-PSA", transform=ax[2].transAxes)
ax[3].text(0.02, 0.90, "d. Solar Orbiter/SWA-PSA", transform=ax[3].transAxes)
ax[4].text(0.02, 0.90, "e. Solar Orbiter/SWA-PSA", transform=ax[4].transAxes)

plt.tight_layout()

# 4. Computions using SPICE kernals now supported since sunpy 5.1

In [ ]:
from sunpy.data import cache
from sunpy.coordinates import spice, frames

In [ ]:
solo_kernal_urls = [
    "spk/de421.bsp",
    "spk/solo_ANC_soc-orbit-stp_20200210-20301120_280_V1_00288_V01.bsp",
]
solo_kernal_urls = [f"http://spiftp.esac.esa.int/data/SPICE/SOLAR-ORBITER/kernels/{url}"
               for url in solo_kernal_urls]

psp_kernals = ["https://spdf.gsfc.nasa.gov/pub/data/psp/ephemeris/spice/Long_Term_Predicted_Ephemeris/spp_nom_20180812_20250831_v039_RO6.bsp"]

kernals = solo_kernal_urls + psp_kernals

kernel_files = [cache.download(url) for url in kernals]
spice.initialize(kernel_files)

In [ ]:
obstime = Time("2020-03-01") + np.arange(0, 1767, 1)*u.day
solo_spacecraft = spice.get_body('Solar Orbiter', obstime)
psp_spacecraft = spice.get_body('SOLAR PROBE PLUS', obstime)

solo_spacecraft_hgs = solo_spacecraft.heliographic_stonyhurst
psp_spacecraft_hgs = psp_spacecraft.heliographic_stonyhurst

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='polar')
im = ax.scatter(solo_spacecraft_hgs.lon.to(u.rad), solo_spacecraft_hgs.radius.to(u.au), s=2, label="Solar Orbiter")
im2 = ax.scatter(psp_spacecraft_hgs.lon.to(u.rad), psp_spacecraft_hgs.radius.to(u.au), s=2, label="Parker Solar Probe")
ax.set_theta_zero_location("S")
plt.legend()
plt.show()


In [ ]:
fig, ax = plt.subplots(2, sharex=True, figsize=(12, 6))
ax[0].plot(solo_spacecraft_hgs.obstime.datetime, solo_spacecraft_hgs.radius.to(u.AU), label="Solar Orbiter")
ax[0].plot(psp_spacecraft_hgs.obstime.datetime, psp_spacecraft_hgs.radius.to(u.AU), label="PSP")

ax[1].plot(solo_spacecraft_hgs.obstime.datetime, solo_spacecraft_hgs.lat.to(u.deg), label="Solar Orbiter")
ax[1].plot(psp_spacecraft_hgs.obstime.datetime, psp_spacecraft_hgs.lat.to(u.deg), label="PSP")

ax[0].set_xlim(psp_spacecraft_hgs.obstime.datetime[0], psp_spacecraft_hgs.obstime.datetime[-1])




ax[0].set_xlim(psp_spacecraft_hgs.obstime.datetime[0], psp_spacecraft_hgs.obstime.datetime[-1])
ax[0].set_ylabel("Distance to Sun (AU)")
ax[1].set_ylabel("Solar Latitude (deg)")
ax[1].set_xlabel("Time")
ax[0].legend()
ax[1].legend()
plt.tight_layout()
plt.subplots_adjust(hspace=0.05)
